## Operator and function overloading
__________________________________

Alex Martelli: **duck typing** 

- don’t check whether it **is-a** duck -- check whether it **quacks-like-a** duck, **walks-like-a** duck, ... 
- depending on exactly what **subset** of duck-like behavior you need to play your language-games with 



## 1. Standard methods of the class as implementation of the static polymorphism concept in Python
_______________________
* **standard methods** as the subset of class protocol 
* for doing standard operation on instances of some class the corresponding **dunder methods** of this class can be overloaded 
* if there is a built-in function ``func()`` and the corresponding special method for the function is ``__func__()``, Python interprets a call to the function ``func(obj)`` as ``obj.__func__()`` (as usual -- ``len`` is exception)
* in the case of operator `` opr`` and the corresponding special method for it `` __opr__()`` Python interprets  ``obj1 <opr> obj2`` as ``obj1.__opr__(obj2)``
* more often than not, the special method call is **implicit** (the only special method that is frequently called by user code directly is ``__init__``, to invoke the initializer of the superclass )
* setting a special method to ``None`` indicates that the corresponding operation is **not available** (for example, if a class sets ``__iter__()`` to ``None``, the class is not iterable, so calling ``iter()`` on its instances will raise a ``TypeError``) 

Python strikes a good balance between flexibility, usability and safety by imposing some limitations:
* operators for the built-in types can't be overload
* only existing operators can be overload 
* few operators ``is, and, or, not`` can’t be overloaded

#### 1.1. Object representations
___________________

* ``__repr__`` and ``__str__`` -- to support ``repr()`` and ``str()``  
* ``__bytes__`` and ``__format__`` -- to support alternate representations of objects:
  * ``__bytes__``  is analogous to ``__str__``: it’s called by ``bytes()`` to get the object represented as a byte sequence 
  * ``__format__`` -- using by ``format()`` and ``str.format()`` to get string displays of objects using special formatting codes

Corey Schafer [Special (Magic/Dunder) Methods](https://www.youtube.com/watch?v=3ohzBxoFHAY&list=PL-osiE80TeTt2d9bfVyTiXJA-UTHn6WwU&index=45&t=0s&ab_channel=CoreySchafer)

In [ ]:
class Employee:
    """ v.1.0 -- A simple class"""
    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.email = first + '.' + last + '@email.com'
        self.pay = pay
    
    def fullname(self):
        return f'{self.first} {self.last}'

In [ ]:
emp_1 = Employee('Corey', 'Schafer', 50000)
emp_2 = Employee('Test', 'Employee', 60000)

In [ ]:
print(emp_1)

In [ ]:
emp_1.__str__()

In [ ]:
emp_1.__repr__()

In [ ]:
class Employee:
    """ v.1.1 -- A simple class + repr"""
    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.email = first + '.' + last + '@email.com'
        self.pay = pay
    
    def fullname(self):
        return f'{self.first} {self.last}'
    
    def __repr__(self):
        return f'Employee({self.first}, {self.last}, {self.pay})'
    
emp_1 = Employee('Corey', 'Schafer', 50000)

In [ ]:
emp_1.__repr__()

In [ ]:
repr(emp_1)

In [ ]:
print(emp_1)

In [ ]:
class Employee:
    """ v.1.2 --  A simple class + repr + str"""
    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.email = first + '.' + last + '@email.com'
        self.pay = pay
    
    def fullname(self):
        return f'{self.first} {self.last}'
    
    def __repr__(self):
        return f'Employee({self.first}, {self.last}, {self.pay})'
    
    def __str__(self):
        return f'{self.fullname()} -- {self.email}'
    
       
emp_1 = Employee('Corey', 'Schafer', 50000)

In [ ]:
print(emp_1)

In [ ]:
str(emp_1)

In [ ]:
repr(emp_1)

#### 1.2. Operator overloading for built-in types 
______________________

In [ ]:
print(1+2)

In [ ]:
print(int.__add__(1, 2))

In [ ]:
print(str.__add__('ajhsgv', 'b'))

#### 1.3. Operator overloading for simple class -- [Emulating numeric types](https://docs.python.org/3/reference/datamodel.html#object.__add__)
________________________
It is important that the emulation **only be implemented** to the degree that it makes sense for the object being modelled

In [ ]:
class Employee:
    """v.2.0 --  A simple class+ repr + str + add"""
    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.email = first + '.' + last + '@email.com'
        self.pay = pay
    
    def fullname(self):
        return f'{self.first} {self.last}'
    
    def __repr__(self):
        return f'Employee({self.first}, {self.last}, {self.pay})'
    
    def __str__(self):
        return f'{self.fullname()} - {self.email}'
    
    def __add__(self, other):
        return self.pay+other.pay
    
    
emp_1 = Employee('Corey', 'Schafer', 50000)
emp_2 = Employee('Test', 'Employee', 60000)

In [ ]:
print(emp_1+emp_2)

## 2. A sliceable sequence
_________________
Any class that implements  the ``__len__`` and ``__getitem__`` methods with the standard signature and semantics can be used anywhere a sequence is expected

#### 2.1. Operator overloading for any sequences
______________________

In [ ]:
class MySeq:
    def __getitem__(self, index):
        return index 
s = MySeq()
s[5] 

In [ ]:
len(s)

In [ ]:
s[1:4]

In [ ]:
type(s[1:4])

In [ ]:
s[1:4:2]

In [ ]:
s[1:4:2, 9]

In [ ]:
s[1:4:2, 7:9]

In [ ]:
dir(slice)

* ``slice`` is a built-in type 
* there are data attributes ``start``, ``stop`` and ``step`` and an ``indices`` method in ``slice``

In [ ]:
help(slice.indices)

``indices`` exposes the logic that’s implemented in the built-in sequences
* to gracefully handle missing or negative indices and slices that are longer than the target sequence
* to produces “normalized” tuples of non-negative ``start``, ``stop`` and ``stride`` integers adjusted to fit within the bounds of a sequence of the given length

#### 2.2. Operator overloading for numerical sequences
______________________

In [ ]:
#Luciano Ramalho. Fluent Python
from array import array
import reprlib
import math
import itertools
import numbers

class Vector:
    """v.1.0 --  A simple class with standard set of the operators including
       __len__ and __getitem
    """
    typecode = 'd' #d -- decimal

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components) # to allow iteration

    def __repr__(self):
        components = reprlib.repr(self._components) # to get a limited-length representation
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self) 

    def __pos__(self):
        return Vector(self)   

    def __bool__(self):
        return bool(abs(self))
    
    #-----------------------------------

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # new Vector instance
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))
    
    #---------------------------------------
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented


In [ ]:
v7=Vector(range(7))
v7

In [ ]:
v7[1:4]

In [ ]:
v7[1,2]

In [ ]:
v2=Vector([3.1, 4.2])
v2

In [ ]:
v=Vector((0, 1, 2))
v

In [ ]:
Vector(range(10))

In [ ]:
str(Vector(range(10)))

In [ ]:
bv=bytes(v)
bv

In [ ]:
vB=Vector.frombytes(bv)
vB

In [ ]:
v3=Vector(range(3))
v3

In [ ]:
v2+v3

In [ ]:
v==v3 

In [ ]:
v==v 

In [ ]:
v is v

In [ ]:
v is v3

In [ ]:
len(v3)

#### 2.3. Implementation of reverse operators
_____________________

In [ ]:
v+(10,20,30)#itertools.zip_longest uses any iterables

In [ ]:
(10,20,30)+v

![add_radd](add_radd.jpg)

In [ ]:
from array import array
import reprlib
import math
import itertools
import numbers

class Vector:
    """v.2.0 --  A simple class with standard set of the operators including
       __radd__ 
    """
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components) # to allow iteration

    def __repr__(self):
        components = reprlib.repr(self._components) # to get a limited-length representation
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>

    def __bool__(self):
        return bool(abs(self))

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
    
    #------------------------------------

    def __radd__(self, other):
        return self + other

In [ ]:
help(itertools.zip_longest)

In [ ]:
v=Vector((0, 1, 2))
v

In [ ]:
(10,20,30)+v

In [ ]:
(10,20,30,40)+v

## 3. Dynamic attribute access
________________

*  ``__getattr__`` -- invoked by the interpreter when attribute lookup fails

**Lookup** for an attribute named ``x`` given by ``my_obj.x``:

1. Python checks if the ``my_obj`` instance has ``x``
2. search goes to the class ``my_obj.__class__``
3. up the inheritance graph

If `` x`` is not found, then ``my_obj.__getattr__()``  is called with ``self`` and the name of the attribute as a string

In [ ]:
from array import array
import reprlib
import math

class Vector:
    """v.3.0 --  A simple class with standard set of the operators including
       __getattr__ 
    """
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other
    
    shortcut_names = 'xyzt'
    
    # ------------------------------------------------
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

In [ ]:
v=Vector(range(5))

In [ ]:
v

In [ ]:
dir(v)

In [ ]:
v.x

In [ ]:
dir(v)

In [ ]:
v.x=10

In [ ]:
v.x

In [ ]:
dir(v)

## 4. Read-only dynamic access 
________________

In [ ]:
v

In [ ]:
from array import array
import reprlib
import math

class Vector:
    """v.3.0 --  A simple class with standard set of the operators including
       __setattr__ 
    """
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))#all(iterable, /) Return True if bool(x) is True for all values x in the iterable.

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0) # hash_k ^ hash_{k+1}

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other
    
    
    #---------------------------------------
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name):
        """ getting the atribute by shortcut name"""
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        """ setting atributes with shortcut name readonly"""
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

In [ ]:
v=Vector(range(5))

In [ ]:
v.x

In [ ]:
v.x = 1.1

In [ ]:
v.x

In [ ]:
v.ab=1.1

In [ ]:
v

In [ ]:
v.ab

* We are not disallowing setting all attributes, only single-letter, lowercase ones, to avoid confusion with the supported read-only attributes x, y, z and t
* Very often when you implement ``__getattr__`` you need to code ``__setattr__`` as well, to avoid inconsistent behavior in your objects